In [1]:
#notebook for scraping
#dependencies 

import os 
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager


In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430
[WDM] - Driver [/Users/summyabakshi/.wdm/drivers/chromedriver/mac64/90.0.4430.24/chromedriver] found in cache


In [3]:
#website to scrape 
url = 'https://redplanetscience.com/'
#retrieve page wiht splinter module - this page will need to stay open to retreive the parsed text
browser.visit(url)


In [4]:
html = browser.html
#parse with soup
soup = BeautifulSoup(html, 'html.parser')

#### Scrape the Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [5]:
#parent divs for latest article title and parargaraph

title = soup.find('div', class_='content_title').text
print(title)
para = soup.find('div', class_='article_teaser_body').text
print(para)



NASA's New Mars Rover Is Ready for Space Lasers
Perseverance is one of a few Mars spacecraft carrying laser retroreflectors. The devices could provide new science and safer Mars landings in the future.


#### JPL Mars Space Images - Featured Image

In [6]:
jpl_url = 'https://spaceimages-mars.com/'
#open the website for scraping
browser.visit(jpl_url)


In [7]:
html = browser.html
#parse
soup = BeautifulSoup(html, 'html.parser')


In [8]:
featured_img_url = soup.find('img', class_="headerimage fade-in")['src']
featured_img_url = (jpl_url + featured_img_url)
print(featured_img_url)

https://spaceimages-mars.com/image/featured/mars3.jpg


### Pandas Scraping 

#### Mars Facts

In [9]:
mars_facts_url = 'https://galaxyfacts-mars.com/'
#create a tablular structure out of the URL
table = pd.read_html(mars_facts_url)
table

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [10]:
facts_df = table[0]
facts_df.head()

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"


In [11]:
facts_df = facts_df.rename(columns={0:'Mars-Earth Comparison', 1: 'Mars', 2:'Earth' })
facts_df

,Mars-Earth Comparison,Mars,Earth
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [12]:
facts_df = facts_df.drop([0])


In [13]:
facts_df = facts_df.set_index(['Mars-Earth Comparison'])
facts_df.head()

,Mars,Earth
Mars-Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days


In [14]:
#convert table to html string
html_table = facts_df.to_html('mars_facts_table.html')
html_table
#do they want us to export the HTML as well ?

#### Mars Hemispheres

In [15]:
#hesmisphere website for scraping
hemi_url = 'https://marshemispheres.com/'
#retreive page with requests module 
browser.visit(hemi_url)
#create soup object with parser
html = browser.html
#parse
soup = BeautifulSoup(html, 'html.parser')

hemi = soup.find_all('div', class_ = 'description')
print(hemi)

[<div class="description">
<a class="itemLink product-item" href="cerberus.html">
<h3>Cerberus Hemisphere Enhanced</h3>
</a>
<span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/>
<p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p>
</div>, <div class="description">
<a class="itemLink product-item" href="schiaparelli.html">
<h3>Schiaparelli Hemisphere Enhanced</h3>
</a>
<span class="subtitle" style="float:left">image/tiff 35 MB</span><span class="pubDate" style="float:right"></span><br/>
<p>Mosaic of the Schiaparelli hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. The images were acquired in 1980 during early northern…</p>
</div>, <div class="description">
<a class="itemLink product-item" href="syrtis.ht

In [19]:
hem_list = []

for i in range(len(hemi)):
    
    hemi_link = browser.find_by_css('a.product-item h3')
    hemi_link[i].click()
    #time.sleep(1)
    
    img_html = browser.html
    img_soup = BeautifulSoup(img_html, 'html.parser')
    
    img_title = img_soup.find('h2', class_="title").text
    
    img_find_class = img_soup.find('div', class_='downloads')
    img_find_click = img_find_class.find('li')
    img_find = img_find_click.find('a')['href']
    
    img_url = f'{hemi_url}{img_find}'
    
    hem_list.append({'title': img_title,
                    'img_url': img_url})
    
    browser.back()
hem_list
    

               

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]

In [20]:
browser.quit()

In [21]:
mars_dict = {
    'titles': title,
    'para': para,
    'featured img': featured_img_url, 
    'hemisphere imgs': hem_list 
}
mars_dict

{'titles': "NASA's New Mars Rover Is Ready for Space Lasers",
 'para': 'Perseverance is one of a few Mars spacecraft carrying laser retroreflectors. The devices could provide new science and safer Mars landings in the future.',
 'featured img': 'https://spaceimages-mars.com/image/featured/mars3.jpg',
 'hemisphere imgs': [{'title': 'Cerberus Hemisphere Enhanced',
   'img_url': 'https://marshemispheres.com/images/full.jpg'},
  {'title': 'Schiaparelli Hemisphere Enhanced',
   'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
  {'title': 'Syrtis Major Hemisphere Enhanced',
   'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
  {'title': 'Valles Marineris Hemisphere Enhanced',
   'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]}